# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.groupby(["CustomerID","ProductName"]).sum().reset_index(level=["ProductName"])
df

,ProductName,SalesID,ProductID,Quantity
CustomerID,,,,
33,Apricots - Dried,172592,324,1
33,Assorted Desserts,5363705,219,1
33,Bandage - Flexible Neon,6299165,273,1
33,"Bar Mix - Pina Colada, 355 Ml",3147536,281,1
33,"Beans - Kidney, Canned",6507033,300,1
...,...,...,...,...
98200,Vol Au Vents,5160346,272,50
98200,Wasabi Powder,4438427,80,25
98200,Wine - Fume Blanc Fetzer,6226060,76,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
df1  = pd.pivot_table(df, index="ProductID", values="Quantity", columns="CustomerID", fill_value=0)
df1

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductID,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0.0,0,1.0,0,0,...,0,0,0.0,0,0,0.0,25,0.0,0,25
2,0,0,1,0.0,0.0,0.0,1,0.0,0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,0
3,0,0,0,0.0,0.0,0.0,0,0.0,1,0,...,0,0,0.0,0,0,0.0,0,0.0,0,0
4,0,1,0,0.0,0.0,0.0,1,1.0,0,1,...,25,0,0.0,0,0,0.0,0,0.0,0,0
5,0,0,0,0.0,0.0,0.0,0,0.0,0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,0,0,0.0,0.0,0.0,0,0.0,0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,0
1308,0,0,0,0.0,0.0,0.0,0,0.0,0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,0
1314,0,0,0,0.0,0.0,0.0,0,0.0,0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
from scipy.spatial.distance import pdist, squareform
distances = pdist(df1.T, 'euclidean')
print(distances)
print("-----Squareform-----")
print(squareform(distances))

[ 12.08304597  10.67707825  11.05667219 ... 305.41979307 310.49355871
 307.20514319]
-----Squareform-----
[[  0.          12.08304597  10.67707825 ... 225.06054741 238.79070334
  229.77380181]
 [ 12.08304597   0.          11.74734012 ... 224.43317491 239.24673456
  229.26840166]
 [ 10.67707825  11.74734012   0.         ... 224.5044543  238.89537459
  229.33817824]
 ...
 [225.06054741 224.43317491 224.5044543  ...   0.         305.41979307
  310.49355871]
 [238.79070334 239.24673456 238.89537459 ... 305.41979307   0.
  307.20514319]
 [229.77380181 229.26840166 229.33817824 ... 310.49355871 307.20514319
    0.        ]]


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
similarityList = pd.DataFrame(1/(1 + squareform(pdist(df1.T, 'euclidean'))), 
                         index=df1.columns, columns=df1.columns)
similarityList.head(5)


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.076435,0.085638,0.082942,0.079111,0.081430,0.075025,0.082632,0.079246,0.078977,...,0.004805,0.004669,0.004497,0.005019,0.004304,0.004596,0.004578,0.004424,0.004170,0.004333
200,0.076435,1.000000,0.078448,0.076134,0.072740,0.078382,0.074129,0.077109,0.077674,0.075956,...,0.004829,0.004676,0.004496,0.005041,0.004311,0.004603,0.004602,0.004436,0.004162,0.004343
264,0.085638,0.078448,1.000000,0.080848,0.082251,0.082327,0.075025,0.081430,0.083333,0.079518,...,0.004822,0.004679,0.004488,0.005035,0.004316,0.004616,0.004595,0.004435,0.004168,0.004341
356,0.082942,0.076134,0.080848,1.000000,0.079861,0.078579,0.073425,0.086867,0.077865,0.077610,...,0.004816,0.004676,0.004498,0.005025,0.004318,0.004606,0.004568,0.004434,0.004172,0.004344
412,0.079111,0.072740,0.082251,0.079861,1.000000,0.078777,0.074911,0.082479,0.077547,0.076800,...,0.004805,0.004688,0.004495,0.005035,0.004314,0.004613,0.004588,0.004432,0.004174,0.004345


In [8]:
similarities = similarityList[33].sort_values(ascending=False)[1:6]
top5 = similarities.index
top5

Int64Index([3535, 1072, 264, 3305, 3074], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
df5 = df.loc[top5]
df5

,ProductName,SalesID,ProductID,Quantity
CustomerID,,,,
3535,Anchovy Paste - 56 G Tube,3904179,131,1
3535,Apricots - Dried,4143868,324,1
3535,Bacardi Breezer - Tropical,2311303,49,1
3535,Bandage - Fexible 1x3,5361156,93,1
3535,"Beef - Chuck, Boneless",2496943,439,1
...,...,...,...,...
3074,Wine - Crozes Hermitage E.,2453132,51,1
3074,Wine - Ej Gallo Sierra Valley,4666469,442,1
3074,Wine - Magnotta - Belpaese,2261766,78,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
df6 = df5.groupby(["ProductName"], as_index=False).sum().sort_values("Quantity", ascending=False)
df6

,ProductName,SalesID,ProductID,Quantity
51,Cod - Black Whole Fillet,12214649,1212,4
205,Wine - Chardonnay South,5988469,580,4
203,Wine - Blue Nun Qualitatswein,13546539,1076,4
210,"Wine - Magnotta, Merlot Sr Vqa",19027984,576,3
142,Pork - Hock And Feet Attached,16356606,597,3
...,...,...,...,...
87,Ice Cream Bar - Drumstick,1330480,154,1
88,Ice Cream Bar - Oreo Cone,5317028,413,1
89,Isomalt,4125106,350,1
90,Jagermeister,5687769,374,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
# Productos comprados por los 5 clientes más parecidos al cliente 33:
productTop5 = df6["ProductName"]

#Productos comprados por el cliente 33:
product33 = df.loc[33]["ProductName"]

listaNoComprados=[]
for e in productTop5:
    if e not in product33 and len(listaNoComprados)<5:
        listaNoComprados.append(e)
               
print(listaNoComprados)

['Cod - Black Whole Fillet', 'Wine - Chardonnay South', 'Wine - Blue Nun Qualitatswein', 'Wine - Magnotta, Merlot Sr Vqa', 'Pork - Hock And Feet Attached']


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [25]:
# Diccionario vacío:
dictionary={}

# Lista de clientes (no repetidos y ordenados):
customers = data.CustomerID
sort_cust = sorted(customers.unique())
sort_cust

[33,
 200,
 264,
 356,
 412,
 464,
 477,
 639,
 649,
 669,
 694,
 756,
 883,
 891,
 1008,
 1034,
 1066,
 1072,
 1336,
 1428,
 1435,
 1534,
 1577,
 1594,
 1754,
 1839,
 1920,
 2187,
 2329,
 2503,
 2556,
 2566,
 2582,
 2617,
 2686,
 2754,
 2776,
 2902,
 2915,
 2939,
 3074,
 3253,
 3267,
 3305,
 3317,
 3472,
 3531,
 3535,
 3544,
 3885,
 3903,
 3909,
 4261,
 4479,
 4595,
 4644,
 4723,
 5104,
 5177,
 5183,
 5224,
 5230,
 5434,
 5678,
 5739,
 5832,
 5921,
 5968,
 5986,
 5997,
 6001,
 6185,
 6317,
 6352,
 6367,
 6447,
 6543,
 6734,
 6817,
 6851,
 7021,
 7134,
 7223,
 7543,
 7606,
 7623,
 7772,
 7827,
 7987,
 7999,
 8316,
 8367,
 8372,
 8539,
 8711,
 8917,
 8962,
 9047,
 9132,
 9260,
 9302,
 9533,
 9546,
 9597,
 9771,
 9804,
 10016,
 10094,
 10114,
 10224,
 10311,
 10418,
 10739,
 11023,
 11057,
 11083,
 11102,
 11253,
 11325,
 11380,
 11441,
 11443,
 11614,
 11628,
 11744,
 11883,
 11933,
 12059,
 12206,
 12232,
 12381,
 12529,
 12561,
 12596,
 12598,
 12852,
 12888,
 13062,
 13118,
 13338,
 

In [49]:
# Iteración (lo hago de 50 para ver que funciona):
for c in sort_cust[:50]:
    # los 5 clientes más parecidos:
    similares5 = similarityList[c].sort_values(ascending=False)[1:6]
    
    # el índice de dichos clientes:
    top5 = similares5.index
    
    # df con el comportamiento de dichos clientes:
    df5 = df.loc[top5]
    
    # df con los productos de los clientes:
    df6 = df5.groupby(["ProductName"], as_index=False).sum().sort_values("Quantity", ascending=False)

    # Productos comprados por los 5 clientes más parecidos al cliente c:
    productTop5 = df6["ProductName"]
    
    # Productos comprados por el cliente c:
    productC = df.loc[c]["ProductName"]
    
    # Lista de los 5 productos recomendables
    listaNoComprados=[]
    for p in productTop5:
        if p not in productC and len(listaNoComprados)<5:
            listaNoComprados.append(p)
    d = {c: listaNoComprados}
    dictionary.update(d)
    
print(dictionary)

{33: ['Cod - Black Whole Fillet', 'Wine - Chardonnay South', 'Wine - Blue Nun Qualitatswein', 'Wine - Magnotta, Merlot Sr Vqa', 'Pork - Hock And Feet Attached'], 200: ['Pork - Kidney', 'Wine - Gato Negro Cabernet', 'Smirnoff Green Apple Twist', 'Lettuce - Treviso', 'Ezy Change Mophandle'], 264: ['Corn Meal', 'Pepper - Black, Whole', 'Wine - Blue Nun Qualitatswein', 'Fish - Scallops, Cold Smoked', 'Lamb - Ground'], 356: ['Olive - Spread Tapenade', 'Fenngreek Seed', 'Tart Shells - Sweet, 4', 'Sprouts - Baby Pea Tendrils', 'Wine - Magnotta - Belpaese'], 412: ['Soup - Campbells Bean Medley', 'Vol Au Vents', 'Wine - Blue Nun Qualitatswein', 'Muffin - Carrot Individual Wrap', 'Pepper - Black, Whole'], 464: ['Tea - Decaf Lipton', 'Pepper - Black, Whole', 'Ketchup - Tomato', 'Knife Plastic - White', 'Sauce - Hollandaise'], 477: ['Bar - Granola Trail Mix Fruit Nut', 'Ketchup - Tomato', 'Cheese - Cambozola', 'Yoghurt Tubes', 'Sausage - Breakfast'], 639: ['Lamb - Ground', 'Sardines', 'Wine - Ej G

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [50]:
record = pd.DataFrame(dictionary)
record

,33,200,264,356,412,464,477,639,649,669,...,48611,48641,48642,48655,48818,48858,48963,49005,49159,49454
0,Cod - Black Whole Fillet,Pork - Kidney,Corn Meal,Olive - Spread Tapenade,Soup - Campbells Bean Medley,Tea - Decaf Lipton,Bar - Granola Trail Mix Fruit Nut,Lamb - Ground,Sardines,Butter - Unsalted,...,Wine - Chardonnay South,Coffee - Dark Roast,Flour - Whole Wheat,"Oil - Shortening,liqud, Fry",Mussels - Frozen,Chicken - Soup Base,Assorted Desserts,Cornflakes,Wiberg Super Cure,"Pasta - Penne, Rigate, Dry"
1,Wine - Chardonnay South,Wine - Gato Negro Cabernet,"Pepper - Black, Whole",Fenngreek Seed,Vol Au Vents,"Pepper - Black, Whole",Ketchup - Tomato,Sardines,Wine - Ej Gallo Sierra Valley,Bar - Granola Trail Mix Fruit Nut,...,Beef - Striploin Aa,Meldea Green Tea Liquor,Browning Caramel Glace,Lamb - Ground,Soup Knorr Chili With Beans,Wine - Blue Nun Qualitatswein,Sun - Dried Tomatoes,Onions - Cippolini,Beer - Sleemans Cream Ale,Spinach - Baby
2,Wine - Blue Nun Qualitatswein,Smirnoff Green Apple Twist,Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Wine - Blue Nun Qualitatswein,Ketchup - Tomato,Cheese - Cambozola,Wine - Ej Gallo Sierra Valley,Bar - Granola Trail Mix Fruit Nut,Sausage - Breakfast,...,Eggplant - Asian,Appetizer - Sausage Rolls,Lambcasing,Cumin - Whole,Coffee - Hazelnut Cream,Pork - Inside,Coffee - Dark Roast,Lettuce - Spring Mix,"Pork - Bacon, Double Smoked",Assorted Desserts
3,"Wine - Magnotta, Merlot Sr Vqa",Lettuce - Treviso,"Fish - Scallops, Cold Smoked",Sprouts - Baby Pea Tendrils,Muffin - Carrot Individual Wrap,Knife Plastic - White,Yoghurt Tubes,"Fish - Scallops, Cold Smoked","Veal - Inside, Choice","Nut - Chestnuts, Whole",...,Yoghurt Tubes,Beets - Mini Golden,"Doilies - 5, Paper",Campari,Grouper - Fresh,"Nut - Chestnuts, Whole",Cheese - Mix,Cassis,Island Oasis - Mango Daiquiri,Phyllo Dough
4,Pork - Hock And Feet Attached,Ezy Change Mophandle,Lamb - Ground,Wine - Magnotta - Belpaese,"Pepper - Black, Whole",Sauce - Hollandaise,Sausage - Breakfast,Soupfoamcont12oz 112con,Butter - Unsalted,"Beets - Candy Cane, Organic",...,Cookie - Dough Variety,"Pepsi - Diet, 355 Ml",Pastry - Raisin Muffin - Mini,"Wine - White, Colubia Cresh",Coffee - Dark Roast,Tuna - Salad Premix,Chips Potato Salt Vinegar 43g,Halibut - Steaks,"Water, Tap",Sea Bass - Whole


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [51]:
distances = pdist(df1.T, 'cityblock')
dictionary2 = {}

# Iteración:
for c in sort_cust[:50]:
    # los 5 clientes más parecidos:
    similares5 = similarityList[c].sort_values(ascending=False)[1:6]
    # el índice de dichos clientes:
    top5 = similares5.index
    
    # df con el comportamiento de dichos clientes:
    df5 = df.loc[top5]
    
    # df con los productos de los clientes:
    df6 = df5.groupby(["ProductName"], as_index=False).sum().sort_values("Quantity", ascending=False)

    # Productos comprados por los 5 clientes más parecidos al cliente c:
    productTop5 = df6["ProductName"]
    
    # Productos comprados por el cliente c:
    productC = df.loc[c]["ProductName"]
    
    # Lista de los 5 productos recomendables
    listaNoComprados=[]
    for p in productTop5:
        if p not in productC and len(listaNoComprados)<5:
            listaNoComprados.append(p)
    d = {c: listaNoComprados}
    dictionary2.update(d)
    
print(dictionary2)

{33: ['Cod - Black Whole Fillet', 'Wine - Chardonnay South', 'Wine - Blue Nun Qualitatswein', 'Wine - Magnotta, Merlot Sr Vqa', 'Pork - Hock And Feet Attached'], 200: ['Pork - Kidney', 'Wine - Gato Negro Cabernet', 'Smirnoff Green Apple Twist', 'Lettuce - Treviso', 'Ezy Change Mophandle'], 264: ['Corn Meal', 'Pepper - Black, Whole', 'Wine - Blue Nun Qualitatswein', 'Fish - Scallops, Cold Smoked', 'Lamb - Ground'], 356: ['Olive - Spread Tapenade', 'Fenngreek Seed', 'Tart Shells - Sweet, 4', 'Sprouts - Baby Pea Tendrils', 'Wine - Magnotta - Belpaese'], 412: ['Soup - Campbells Bean Medley', 'Vol Au Vents', 'Wine - Blue Nun Qualitatswein', 'Muffin - Carrot Individual Wrap', 'Pepper - Black, Whole'], 464: ['Tea - Decaf Lipton', 'Pepper - Black, Whole', 'Ketchup - Tomato', 'Knife Plastic - White', 'Sauce - Hollandaise'], 477: ['Bar - Granola Trail Mix Fruit Nut', 'Ketchup - Tomato', 'Cheese - Cambozola', 'Yoghurt Tubes', 'Sausage - Breakfast'], 639: ['Lamb - Ground', 'Sardines', 'Wine - Ej G

In [52]:
record2 = pd.DataFrame(dictionary2)
record2

,33,200,264,356,412,464,477,639,649,669,...,3074,3253,3267,3305,3317,3472,3531,3535,3544,3885
0,Cod - Black Whole Fillet,Pork - Kidney,Corn Meal,Olive - Spread Tapenade,Soup - Campbells Bean Medley,Tea - Decaf Lipton,Bar - Granola Trail Mix Fruit Nut,Lamb - Ground,Sardines,Butter - Unsalted,...,Butter - Unsalted,Lettuce - Treviso,Wine - Blue Nun Qualitatswein,Soup - Campbells Bean Medley,Bar - Granola Trail Mix Fruit Nut,"Pepper - Black, Whole",Knife Plastic - White,Butter - Unsalted,Cheese - Cambozola,"Pepper - Black, Whole"
1,Wine - Chardonnay South,Wine - Gato Negro Cabernet,"Pepper - Black, Whole",Fenngreek Seed,Vol Au Vents,"Pepper - Black, Whole",Ketchup - Tomato,Sardines,Wine - Ej Gallo Sierra Valley,Bar - Granola Trail Mix Fruit Nut,...,Olive - Spread Tapenade,"Tart Shells - Sweet, 4","Pepper - Black, Whole",Butter - Unsalted,Sausage - Breakfast,Ketchup - Tomato,"Sole - Dover, Whole, Fresh","Nut - Chestnuts, Whole","Fish - Scallops, Cold Smoked",Ketchup - Tomato
2,Wine - Blue Nun Qualitatswein,Smirnoff Green Apple Twist,Wine - Blue Nun Qualitatswein,"Tart Shells - Sweet, 4",Wine - Blue Nun Qualitatswein,Ketchup - Tomato,Cheese - Cambozola,Wine - Ej Gallo Sierra Valley,Bar - Granola Trail Mix Fruit Nut,Sausage - Breakfast,...,"Peas - Pigeon, Dry","Veal - Inside, Choice",Soup - Campbells Bean Medley,"Pepper - Black, Whole",Yoghurt Tubes,Butter - Unsalted,Sauce - Hollandaise,Bar - Granola Trail Mix Fruit Nut,Sprouts - Baby Pea Tendrils,Butter - Unsalted
3,"Wine - Magnotta, Merlot Sr Vqa",Lettuce - Treviso,"Fish - Scallops, Cold Smoked",Sprouts - Baby Pea Tendrils,Muffin - Carrot Individual Wrap,Knife Plastic - White,Yoghurt Tubes,"Fish - Scallops, Cold Smoked","Veal - Inside, Choice","Nut - Chestnuts, Whole",...,Beef - Montreal Smoked Brisket,Skirt - 29 Foot,"Veal - Inside, Choice",Ketchup - Tomato,Butter - Unsalted,Olive - Spread Tapenade,"Pepper - Black, Whole",Soup - Campbells Bean Medley,Wine - Two Oceans Cabernet,Olive - Spread Tapenade
4,Pork - Hock And Feet Attached,Ezy Change Mophandle,Lamb - Ground,Wine - Magnotta - Belpaese,"Pepper - Black, Whole",Sauce - Hollandaise,Sausage - Breakfast,Soupfoamcont12oz 112con,Butter - Unsalted,"Beets - Candy Cane, Organic",...,Pomello,Smirnoff Green Apple Twist,Beef Wellington,Knife Plastic - White,"Fish - Scallops, Cold Smoked",Soup - Campbells Bean Medley,Tea - Decaf Lipton,Wine - Blue Nun Qualitatswein,Halibut - Steaks,Soup - Campbells Bean Medley
